### scraping b-ok site (name of books , urls) using selenium , then download all books 

In [2]:
#Importing packages
from selenium import webdriver
import pandas as pd
import os 
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
import pandas as pd
from collections.abc import Iterable
import warnings
# from pymongo import MongoClient
# from collections.abc import Iterable
import time

In [3]:
selenium_driver_path = "C:/chromedriver"

In [4]:
# get unique elements from a list
def unique(data):
    return list(dict.fromkeys(data))

In [5]:
# get selenium driver object
def get_selenium_driver():
    """
    This function returns the selenium driver object.

    Parameters:
        None

    Returns:
        driver: selenium driver object
    """

    driver = webdriver.Chrome(executable_path = selenium_driver_path)

    return driver

In [6]:
# get BeautifulSoup object
def get_soup(url):
    """
    Given the url of a page, this function returns the soup object.
    
    Parameters:
        url: the link to get soup object for
    
    Returns:
        soup: soup object
    """
    driver = get_selenium_driver()

    driver.get(url)

    driver.implicitly_wait(3)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    driver.close()

    return soup

In [7]:
#save to MongoDB
def saveToDB(collection, city_areas_zones_dict):
    
    try:
        print("collection.count_documents({}): ",  collection.count_documents({}))
        resultSet = collection.count_documents({"City":city_areas_zones_dict["City"],"Areas_Zones":city_areas_zones_dict['Areas_Zones']})

        if resultSet > 0:
            print("city_areas_zones_dict is already scrapped >> ", city_areas_zones_dict)
            print("Skip!")
        else:
            print("New city_areas_zones_dict will be scrapped >> ", city_areas_zones_dict)
            print("I will insert it!")
            collection.insert_one(city_areas_zones_dict)
            
    except Exception as e:
        print('Exception : %s' % str(e))

In [8]:
import re

# scrape b-ok website

def scrape_books():
    books_names = []

    for num in range(1,2):
        base_url = "https://b-ok.cc/s/natural%20language%20processing?page="+str(num)

        home_page_soup = get_soup(base_url)

        # get a get_selenium_driver object
        driver = get_selenium_driver()
        #driver.execute_script("return navigator.userAgent")
        driver.get(base_url)
        

        home_page_soup=home_page_soup.find_all("div",{'id':"searchResultBox"})
        links=home_page_soup[0].find_all("a",{"href":re.compile("^/book/")})
        #print(links)
            

        all_link = []
        for link in links:
            part_of_link = link.get("href")
            link = "https://b-ok.cc"+part_of_link
            all_link.append(link)
            all_link = list(set(all_link))
            print("all_link")
            print(len(all_link))

            print(all_link)
        
        for link in all_link:
            page_driver2 = get_selenium_driver()
            page_driver2.get(link)
            page_driver2.find_element_by_class_name("dlButton").click()
            


        
        time.sleep(3)
        print("----------------------------------")
    driver.close()
        


In [32]:
scrape_books()

all_link
1
['https://b-ok.cc/book/3681492/07d64c']
all_link
1
['https://b-ok.cc/book/3681492/07d64c']
all_link
2
['https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/3681492/07d64c']
all_link
2
['https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/3681492/07d64c']
all_link
3
['https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/3711505/debacd', 'https://b-ok.cc/book/3681492/07d64c']
all_link
3
['https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/3711505/debacd', 'https://b-ok.cc/book/3681492/07d64c']
all_link
4
['https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/4984922/b162b3', 'https://b-ok.cc/book/3711505/debacd', 'https://b-ok.cc/book/3681492/07d64c']
all_link
4
['https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/4984922/b162b3', 'https://b-ok.cc/book/3711505/debacd', 'https://b-ok.cc/book/3681492/07d64c']
all_link
5
['https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/3681492/07d64c', 'https://b-ok.cc/book/755716/be52d

WebDriverException: Message: unknown error: Element <a href="#" id="dbx_2841641" class="btn btn-primary dlButton disabled" data-toggle="tooltip" data-autoopen="true" data-placement="top" title="" data-original-title="Link deleted by legal owner (But you can use <a href=&quot;/msgn/tor&quot; target=&quot;_blank&quot; rel=&quot;nofollow&quot;>TOR</a> to download it anyway)" aria-describedby="tooltip284267">...</a> is not clickable at point (143, 410). Other element would receive the click: <div style="margin:30px 0 15px 0;">...</div>
  (Session info: chrome=80.0.3987.122)
  (Driver info: chromedriver=2.35.528161 (5b82f2d2aae0ca24b877009200ced9065a772e73),platform=Windows NT 10.0.17134 x86_64)


# Read A Book

In [112]:
import PyPDF2
from PyPDF2 import PdfFileReader

In [113]:
#Creating a pdf file object
pdf = open("Natural Language Processing and Text Mining by Anne Kao, Steve R. Poteet (z-lib.org).pdf","rb") #read binary file 

# Ask the user to Enter the Page Number 

In [115]:
pagenumber=input("Enter page number : ")

Enter page number : 77


In [117]:
#creating pdf reader object
pdf_reader = PyPDF2.PdfFileReader(pdf)

#checking number of pages in a pdf file
print(pdf_reader.numPages)

#creating a page object
#get the text in the page you choose
page = pdf_reader.getPage(int(pagenumber))

# Please note that the function above doesn’t work for scanned PDFs.

#finally extracting text from the page
print(page.extractText())

#closing the pdf file
pdf.close()

272
5ACaseStudyinNaturalLanguageBasedWeb
Search
GiovanniMarchisio,NavdeepDhillon,JishengLiang,CarstenTusk,Krzysztof
Koperski,ThienNguyen,DanWhite,andLubosPochman
5.1Introduction
Isthereapublicfornaturallanguagebasedsearch?Thisstudy,basedonourexperi-
encewithaWebportal,attemptstoaddresscriticismsonthelackofscalabilityand
usabilityofnaturallanguageapproachestosearch.OursolutionisbasedonInFact
R,anaturallanguagesearchenginethatcombinesthespeedofkeywordsearchwith
thepowerofnaturallanguageprocessing.InFactperformsclauselevelindexing,and
afullspectrumoffunctionalitythatrangesfromBooleankeywordoperators
tolinguisticpatternmatchinginrealtime,whichincluderecognitionofsyntactic
roles,suchassubject/objectandsemanticcategories,suchaspeopleandplaces.A

userofoursearchcannavigateandretrieveinformationbasedonanunderstanding
ofactions,rolesandrelationships.IndevelopingInFact,weportedthefunctional-
ityofadeeptextanalysisplatformtoamodernsearchenginearchitecture.Our
distributedindexingandsearchservicesa

In [118]:
extracted=page.extractText() #exclude the header of the page
body=extracted.splitlines()# splitting the sentences by \n
del body[0]

sentences_list=list()
whole_sentence=""
for each in body:
    whole_sentence=whole_sentence+each+" "

sentences=whole_sentence.split(".")# splitting the sentences by full stop

import wordninja # to split the sentences that doesnot have white spaces into a list of words
list_of_lists=list()
for i in sentences:
    print(i)
    print(" ")
    #wordninja.split(i)
    print(wordninja.split(i))
    list_of_lists.append(wordninja.split(i))
    print(" ")
    print("---------------------------------")
    
final_sentences=list() #putting all words to gather to form a sentences
for each_list in list_of_lists:
    sentence=""
    for each_word in each_list:
        sentence=sentence+" "+each_word
    sentence=sentence+"."
    final_sentences.append(sentence)

long_sentence="" #putting all sentences together to form one long paragraph
for i in final_sentences:
    long_sentence=long_sentence+i

Search GiovanniMarchisio,NavdeepDhillon,JishengLiang,CarstenTusk,Krzysztof Koperski,ThienNguyen,DanWhite,andLubosPochman 5
 
['Search', 'Giovanni', 'March', 'is', 'io', 'Navdeep', 'D', 'hill', 'on', 'Ji', 'sheng', 'Liang', 'Carsten', 'Tusk', 'Krzysztof', 'Koper', 'ski', 'Th', 'ien', 'Nguyen', 'Dan', 'White', 'and', 'Lu', 'bos', 'P', 'och', 'man', '5']
 
---------------------------------
1Introduction Isthereapublicfornaturallanguagebasedsearch?Thisstudy,basedonourexperi- encewithaWebportal,attemptstoaddresscriticismsonthelackofscalabilityand usabilityofnaturallanguageapproachestosearch
 
['1', 'Introduction', 'Is', 'there', 'a', 'public', 'for', 'natural', 'language', 'based', 'search', 'This', 'study', 'based', 'on', 'our', 'ex', 'peri', 'en', 'ce', 'with', 'a', 'Web', 'portal', 'attempts', 'to', 'address', 'criticisms', 'on', 'the', 'lack', 'of', 's', 'cal', 'ability', 'and', 'usability', 'of', 'natural', 'language', 'approaches', 'to', 'search']
 
---------------------------------
O

# Page Before Summarization

In [119]:
print(long_sentence)

 Search Giovanni March is io Navdeep D hill on Ji sheng Liang Carsten Tusk Krzysztof Koper ski Th ien Nguyen Dan White and Lu bos P och man 5. 1 Introduction Is there a public for natural language based search This study based on our ex peri en ce with a Web portal attempts to address criticisms on the lack of s cal ability and usability of natural language approaches to search. Our solution is based on In Fact R a natural language search engine that combines the speed of keyword search with the power of natural language processing. In Fact performs clause level indexing and a full spectrum of functionality that ranges from Boolean keyword operators to linguistic pattern matching in realtime which include recognition of syntactic roles such as subject object and semantic categories such as people and places. A user of our search can navigate and retrieve information based on an understanding of actions roles and relationships. In developing In Fact we ported the functional it y of a de

# Sentence After Summarization by LexRankSummarizer

In [120]:
## Using SUmy package
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

In [121]:
# For Strings
parser=PlaintextParser.from_string(long_sentence,Tokenizer("english"))
# Using LexRank
summarizer = LexRankSummarizer()
#Summarize the document with 4 sentences
summary = summarizer(parser.document,4)
for sentence in summary:
    print(sentence)

Search Giovanni March is io Navdeep D hill on Ji sheng Liang Carsten Tusk Krzysztof Koper ski Th ien Nguyen Dan White and Lu bos P och man 5.
Our solution is based on In Fact R a natural language search engine that combines the speed of keyword search with the power of natural language processing.
In Fact performs clause level indexing and a full spectrum of functionality that ranges from Boolean keyword operators to linguistic pattern matching in realtime which include recognition of syntactic roles such as subject object and semantic categories such as people and places.
Site statistics and user logs demonstrate that a statistically sign i cant segment of the user population is relying on natural language search functionality.
